<center><h2>SSD : Single Shot Multibox Detection</h2></center>

In [1]:
import tensorflow as tf
import numpy as np
import tensorflow.contrib.slim as slim

C:\Users\user\Anaconda3\Anaconda\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# image size

height = 300
width = 300
num_of_channels = 3
num_of_classes = 21          # on Pascal VOC 2007


l2_penalty = 1e-8

In [3]:
image = tf.placeholder(tf.float32, shape = [None, height, width, num_of_channels])

In [4]:
def tensor_info(tensor):
    print(format(tensor.name, '^23') + format(" |   shape = " + str(tensor.shape), '^20'))

In [5]:
def ssd(image, num_of_classes, l2_penalty):
    
    "Conv Layers"
    print("--------------------- Conv Layers ---------------------")
    conv1 = slim.conv2d(image, 64, [3, 3], scope = 'conv1_1')
    conv1 = slim.conv2d(conv1, 64, [3, 3], scope = 'conv1_2')
    tensor_info(conv1)
    pool1 = slim.max_pool2d(conv1, [2, 2], scope = 'pool1', padding = 'SAME') 
    tensor_info(pool1)
    
    conv2 = slim.repeat(pool1, 2, slim.conv2d, 128, [3, 3], scope = 'conv2')
    tensor_info(conv2)
    pool2 = slim.max_pool2d(conv2, [2, 2], scope = 'pool2')
    tensor_info(pool2)
    
    conv3 = slim.repeat(pool2, 3, slim.conv2d, 256, [3, 3], scope = 'conv3')
    tensor_info(conv3)
    pool3 = slim.max_pool2d(conv3, [2, 2], scope = 'pool3', padding = 'SAME')
    tensor_info(pool3)
    
    conv4 = slim.repeat(pool3, 3, slim.conv2d, 512, [3, 3], scope = 'conv4')
    tensor_info(conv4)
    pool4 = slim.max_pool2d(conv4, [2, 2], scope = 'pool4', padding = 'SAME')
    tensor_info(pool4)
    
    conv5 = slim.repeat(pool4, 3, slim.conv2d, 1024, [3, 3], scope = 'conv5')
    tensor_info(conv5)
    
    conv6 = slim.conv2d(conv5, 1024, [3, 3], scope = 'conv6')
    tensor_info(conv6)
    
    conv7 = slim.conv2d(conv6, 1024, [1, 1], scope = 'conv7')
    tensor_info(conv7)
    
    conv8_1 = slim.conv2d(conv7, 256, [1, 1], scope = 'conv8_1')
    conv8_2 = slim.conv2d(conv8_1, 512, [3, 3], stride = 2, scope = 'conv8_2')
    tensor_info(conv8_2)
    
    conv9_1 = slim.conv2d(conv8_2, 128, [1, 1], scope = 'conv9_1')
    conv9_2 = slim.conv2d(conv9_1, 256, [3, 3], stride = 2, scope = 'conv9_2')
    tensor_info(conv9_2)
    
    conv10_1 = slim.conv2d(conv9_2, 128, [1, 1], scope = 'conv10_1')
    conv10_2 = slim.conv2d(conv10_1, 256, [3, 3], stride = 1, padding = 'VALID', scope = 'conv10_2')
    tensor_info(conv10_2)
    
    conv11_1 = slim.conv2d(conv10_2, 128, [1, 1], scope = 'conv11_1')
    conv11_2 = slim.conv2d(conv11_1, 256, [3, 3], stride = 1, padding = 'VALID', scope = 'conv11_2')
    tensor_info(conv11_2)
    
    
    "Extra Feature Layers"
    print("----------------- Extra Feature Layers -----------------")
    classifier_1 = slim.conv2d(conv4, 4 * num_of_classes, [3, 3], weights_regularizer = slim.l2_regularizer(l2_penalty), scope = 'clf_1')
    detector_1 = slim.conv2d(conv4, 4 * 4, [3, 3], weights_regularizer = slim.l2_regularizer(l2_penalty), scope = 'dtt_1')
    tensor_info(classifier_1)
    tensor_info(detector_1)
    
    classifier_2 = slim.conv2d(conv7, 6 * num_of_classes, [3, 3], scope = 'clf_2')
    detector_2 = slim.conv2d(conv7, 6 * 4, [3, 3], scope = 'dtt_2')
    tensor_info(classifier_2)
    tensor_info(detector_2)
    
    classifier_3 = slim.conv2d(conv8_2, 6 * num_of_classes, [3, 3], scope = 'clf_3')
    detector_3 = slim.conv2d(conv8_2, 6 * 4, [3, 3], scope = 'dtt_3')
    tensor_info(classifier_3)
    tensor_info(detector_3)
    
    classifier_4 = slim.conv2d(conv9_2, 6 * num_of_classes, [3, 3], scope = 'clf_4')
    detector_4 = slim.conv2d(conv9_2, 6 * 4, [3, 3], scope = 'dtt_4')
    tensor_info(classifier_4)
    tensor_info(detector_4)
    
    classifier_5 = slim.conv2d(conv10_2, 4 * num_of_classes, [3, 3], scope = 'clf_5')
    detector_5 = slim.conv2d(conv10_2, 4 * 4, [3, 3], scope = 'dtt_5')
    tensor_info(classifier_5)
    tensor_info(detector_5)
    
    classifier_6 = slim.conv2d(conv11_2, 4 * num_of_classes, [3, 3], scope = 'clf_6')
    detector_6 = slim.conv2d(conv11_2, 4 * 4, [3, 3], scope = 'dtt_6')
    tensor_info(classifier_6)
    tensor_info(detector_6)
    
    total_parameters = np.sum([np.prod(var.get_shape().as_list()) for var in tf.trainable_variables()])
    print("\n  => Number of Weights : " + format(total_parameters, ','))

In [6]:
ssd(image, num_of_classes, l2_penalty)

--------------------- Conv Layers ---------------------
    conv1_2/Relu:0      |   shape = (?, 300, 300, 64)
    pool1/MaxPool:0     |   shape = (?, 150, 150, 64)
 conv2/conv2_2/Relu:0   |   shape = (?, 150, 150, 128)
    pool2/MaxPool:0     |   shape = (?, 75, 75, 128)
 conv3/conv3_3/Relu:0   |   shape = (?, 75, 75, 256)
    pool3/MaxPool:0     |   shape = (?, 38, 38, 256)
 conv4/conv4_3/Relu:0   |   shape = (?, 38, 38, 512)
    pool4/MaxPool:0     |   shape = (?, 19, 19, 512)
 conv5/conv5_3/Relu:0   |   shape = (?, 19, 19, 1024)
     conv6/Relu:0       |   shape = (?, 19, 19, 1024)
     conv7/Relu:0       |   shape = (?, 19, 19, 1024)
    conv8_2/Relu:0      |   shape = (?, 10, 10, 512)
    conv9_2/Relu:0      |   shape = (?, 5, 5, 256)
    conv10_2/Relu:0     |   shape = (?, 3, 3, 256)
    conv11_2/Relu:0     |   shape = (?, 1, 1, 256)
----------------- Extra Feature Layers -----------------
     clf_1/Relu:0       |   shape = (?, 38, 38, 84)
     dtt_1/Relu:0       |   shape = (?,